# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model, but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited.

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique
* Implement a K-folds cross validation modeling pipeline for deep learning models
* Apply regularization techniques to improve your model's performance

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab.

In [1]:
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


In [2]:
# Necessary libraries and classes
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict

from keras import models
from keras import layers
from keras import regularizers
from scikeras.wrappers import KerasRegressor

In this lab you'll be working with the *The Lending Club* data.

- Import the data available in the file `'loan_final.csv'`
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column)
- Print the first five rows of the data
- Print the dimensions of the data

In [3]:
# Import the data
url = 'https://raw.githubusercontent.com/joeldmott/dsc-tuning-neural-networks-from-start-to-finish-lab-v2-1/master/loan_final.csv'
data = pd.read_csv(url)

# Drop rows with no target value
data = data[data['total_pymnt'].notna()]

# Print head
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [4]:
# Print the dimensions of data
data.shape
# My notes: this doesn't seem like a 'small' dataset even if it isn't huge, either.

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42.

In [5]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc',
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Preprocessing (Numerical features)

- Fill all missing values in numeric features with their respective means
- Standardize all the numeric features  
- Convert the final results into DataFrames

In [6]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer()

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = ss_X.fit_transform(X_train_cont)

# Transform test data
X_test_scaled = ss_X.transform(X_test_cont)

## Preprocessing (Categorical features)

- Fill all missing values in categorical features with the string `'missing'`
- One-hot encode all categorical features
- Convert the final results into DataFrames


In [29]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat = X_train_cat.fillna('missing')
X_test_cat = X_test_cat.fillna('missing')

# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names_out(input_features=X_train_cat.columns)

#Why am I getting a "Shape of passed values is (29774, 1)" error below?
#WHat's the shape of X_train_ohe right now?
X_train_ohe.shape

(29774, 20)

In [31]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat.fillna(value='missing', inplace=True)
X_test_cat.fillna(value='missing', inplace=True)

# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names_out(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

In [32]:
#print(X_train_ohe)

In [33]:
# Fit and transform the training data
#X_train_categorical = pd.DataFrame(X_train_ohe, columns=cat_columns)
X_train_categorical = pd.DataFrame(X_train_ohe, columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe, columns=cat_columns)

ValueError: Shape of passed values is (29774, 1), indices imply (29774, 20)

In [26]:
X_train_categorical.head()

,"(home_ownership_MORTGAGE, home_ownership_NONE, home_ownership_OTHER, home_ownership_OWN, home_ownership_RENT, verification_status_Not Verified, verification_status_Source Verified, verification_status_Verified, emp_length_1 year, emp_length_10+ years, emp_length_2 years, emp_length_3 years, emp_length_4 years, emp_length_5 years, emp_length_6 years, emp_length_7 years, emp_length_8 years, emp_length_9 years, emp_length_< 1 year, emp_length_missing)"
0,"(0, 0)\t1.0\n (0, 7)\t1.0\n (0, 9)\t1.0"
1,"(0, 0)\t1.0\n (0, 7)\t1.0\n (0, 9)\t1.0"
2,"(0, 4)\t1.0\n (0, 5)\t1.0\n (0, 14)\t1.0"
3,"(0, 0)\t1.0\n (0, 5)\t1.0\n (0, 16)\t1.0"
4,"(0, 0)\t1.0\n (0, 7)\t1.0\n (0, 9)\t1.0"


Run the below cell to combine the numeric and categorical features.

In [ ]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`)

In [ ]:
# Instantiate StandardScaler
ss_y = None

# Fit and transform Y (train)
y_train_scaled = None

# Transform test Y (test)
y_test_scaled = None

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps:

- Create a function that returns a compiled deep learning model
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained
- Call the `cross_val_predict()` function to perform k-fold cross-validation

In the cell below, we've defined a baseline model that returns a compiled Keras models.

In [ ]:
# Define a function that returns a compiled Keras model
def create_baseline_model():

    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD',
                  loss='mse',
                  metrics=['mse'])

    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and:

- Train for 150 epochs
- Set the batch size to 256

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [ ]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = None

Use `cross_val_predict()` to generate cross-validated predictions with:
- 5-fold cv
- scaled input (`X_train_all`) and output (`y_train_scaled`)

In [ ]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = None

- Find the RMSE on train data

In [ ]:
# RMSE on train data (scaled)


- Convert the scaled predictions back to original scale
- Calculate RMSE in the original units with `y_train` and `baseline_preds`

In [ ]:
# Convert the predictions back to original scale
baseline_preds = None

# RMSE on train data (original scale)


## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [ ]:
# Define a function that returns a compiled Keras model
def create_bigger_model():

    pass

In [ ]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = None

In [ ]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = None

In [ ]:
# RMSE on train data (scaled)


## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting.

In [ ]:
# Define a function that returns a compiled Keras model
def create_regularized_model():

    pass

In [ ]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = None

In [ ]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = None

In [ ]:
# RMSE on train data (scaled)


## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)!

In [ ]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods.